#### imports 

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

def request_bot(meeting_url,language=None):
    native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
    meeting_info = client.request_bot(
        platform="google_meet",
        native_meeting_id=native_meeting_id,
        bot_name="Vexa",
        language=language,
        task = 'transcribe'
    )
    return meeting_info
def get_transcript(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    try:
        for _ in range(10):
            
                transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
                df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
                clear_output()
                display(df.sort_values('absolute_start_time').tail(10))
                time.sleep(1)

    except Exception as e:
        print(e)

def stop_bot(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


#### agenda

#### single bot per user/meeting
#### concurrent_bots
1. create user with 2 concurrency
2. start 1 bot
3. request another bot for the same meeting immediately -> must fail
4. first bot -> must come, admit the bot
5. speak and test transcription
6. launch bot for another meeting, admit the bot, check trnascription
7. launch another bot for the third meeting -> must fail
8. stop bot meeting 2 and and launch bot 3, but come

#### stop bot works for requested and active meetings
1 start meeting 1, stop immediately, start meeting 2 ->bot must come


#### tests

In [3]:
meeting_urls = ["https://meet.google.com/xdy-txjo-fyh", 
            "https://meet.google.com/fjp-cbnn-hug",
            "https://meet.google.com/wjf-bhmw-bix"]

##### 1. create user with 2 concurrency

In [4]:

new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", 
                                    name="test",
                                    max_concurrent_bots=2)
user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key)


DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '28365@example.com', 'name': 'test', 'max_concurrent_bots': 2}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:08 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '145', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"28365@example.com","name":"test","image_url":null,"max_concurrent_bots":2,"data":{},"id":282,"created_at":"2025-09-19T18:08:09.255479"}...

DEBUG: Making POST request to http://localhost:18056/admin/users/282/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:08 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvic

In [5]:
#2. start 1 bot
request_bot(meeting_urls[0])
#3. request another bot for the same meeting immediately (must fail)
try:request_bot(meeting_urls[0])
except Exception as e:
    print(e) #An active or requested meeting already exists for this platform and meeting ID


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xdy-txjo-fyh', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:08 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":631,"user_id":282,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"requested","bot_container_id":"337fc56430453f69052ae853777882802e2e0cd6d22295172d8b62cd40019909","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-19T18:08:09.397069","updated_at":"2025-09-19T18:08:09.420857"}...

DEBUG: Making POST request to http://localhost:18056/bo

4. first bot must come, admit the bot and speak

In [6]:
#time.sleep(15)
#5. speak and test transcription
get_transcript(meeting_urls[0])

'absolute_start_time'


In [7]:
#6. launch bot for another meeting, admit the bot, check trnascription
request_bot(meeting_urls[1])



DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'fjp-cbnn-hug', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:09 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":632,"user_id":282,"platform":"google_meet","native_meeting_id":"fjp-cbnn-hug","constructed_meeting_url":"https://meet.google.com/fjp-cbnn-hug","status":"requested","bot_container_id":"72f2fb591f467e4f3407fc490181a2884ab99ee0c2afedd30b84f736c5a6b562","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-19T18:08:09.735150","updated_at":"2025-09-19T18:08:09.748570"}...


{'id': 632,
 'user_id': 282,
 'platform': 'google_meet',
 'native_meeting_id': 'fjp-cbnn-hug',
 'constructed_meeting_url': 'https://meet.google.com/fjp-cbnn-hug',
 'status': 'requested',
 'bot_container_id': '72f2fb591f467e4f3407fc490181a2884ab99ee0c2afedd30b84f736c5a6b562',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-19T18:08:09.735150',
 'updated_at': '2025-09-19T18:08:09.748570'}

In [8]:
#time.sleep(15)
get_transcript(meeting_urls[1])

'absolute_start_time'


In [9]:
#7. launch another bot for the third meeting -> must fail
#time.sleep(1)
try:request_bot(meeting_urls[2])
except Exception as e:
    print(e) #User has reached the maximum concurrent bot limit (2).


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 403
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:09 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"detail":"User has reached the maximum concurrent bot limit (2)."}...
HTTP Error 403 for POST http://localhost:18056/bots: User has reached the maximum concurrent bot limit (2).


In [10]:
meeting_urls

['https://meet.google.com/xdy-txjo-fyh',
 'https://meet.google.com/fjp-cbnn-hug',
 'https://meet.google.com/wjf-bhmw-bix']

In [11]:
#8. stop bot meeting 2 and and launch bot 3, but come
#time.sleep(5)
stop_bot(meeting_urls[1])
request_bot(meeting_urls[2])


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:09 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '80', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted; meeting finalized immediately (pre-active)."}...

DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}


DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:09 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":633,"user_id":282,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"requested","bot_container_id":"052ac63ff83ca3e448f48969a854e9d6e2bbe674feca0da02771fca9a4de5b01","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-19T18:08:10.029954","updated_at":"2025-09-19T18:08:10.037359"}...


{'id': 633,
 'user_id': 282,
 'platform': 'google_meet',
 'native_meeting_id': 'wjf-bhmw-bix',
 'constructed_meeting_url': 'https://meet.google.com/wjf-bhmw-bix',
 'status': 'requested',
 'bot_container_id': '052ac63ff83ca3e448f48969a854e9d6e2bbe674feca0da02771fca9a4de5b01',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-19T18:08:10.029954',
 'updated_at': '2025-09-19T18:08:10.037359'}

In [12]:
#time.sleep(15)
get_transcript(meeting_urls[2])

'absolute_start_time'


In [13]:
running_bots = client.get_running_bots_status()
len(running_bots) #should be 2



DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:08:09 GMT, Fri, 19 Sep 2025 18:08:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1317', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"052ac63ff83ca3e448f48969a854e9d6e2bbe674feca0da02771fca9a4de5b01","container_name":"vexa-bot-633-30961214","platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","status":"Up Less than a second","normalized_status":"Up","created_at":"2025-09-19T18:08:10+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"282"},"meeting_id_from_name":"633"},{"container_id":"72f2fb591f467e4f3407fc490181a2884...


3

In [46]:
meetings = client.get_meetings()
[m['status'] for m in meetings]


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:30:52 GMT, Fri, 19 Sep 2025 18:30:52 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '2477', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":633,"user_id":282,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"completed","bot_container_id":"vexa-bot-633-30961214","start_time":"2025-09-19T18:08:53.189535","end_time":"2025-09-19T18:13:02.774969","data":{"completion_reason":"stopped","status_transition":[{"to":"joining","from":"requested","source":"bot_callback","timestamp":"2025-09-19T18:08:13.141719"},{"to":"awaiting_admission","from...


['completed', 'completed', 'completed']

In [47]:
meetings

[{'id': 633,
  'user_id': 282,
  'platform': 'google_meet',
  'native_meeting_id': 'wjf-bhmw-bix',
  'constructed_meeting_url': 'https://meet.google.com/wjf-bhmw-bix',
  'status': 'completed',
  'bot_container_id': 'vexa-bot-633-30961214',
  'start_time': '2025-09-19T18:08:53.189535',
  'end_time': '2025-09-19T18:13:02.774969',
  'data': {'completion_reason': 'stopped',
   'status_transition': [{'to': 'joining',
     'from': 'requested',
     'source': 'bot_callback',
     'timestamp': '2025-09-19T18:08:13.141719'},
    {'to': 'awaiting_admission',
     'from': 'joining',
     'source': 'bot_callback',
     'timestamp': '2025-09-19T18:08:22.779233'},
    {'to': 'active',
     'from': 'awaiting_admission',
     'source': 'bot_callback',
     'timestamp': '2025-09-19T18:08:53.185960'},
    {'to': 'completed',
     'from': 'active',
     'reason': 'self_initiated_leave',
     'source': 'user',
     'exit_code': 0,
     'timestamp': '2025-09-19T18:13:02.760822',
     'completion_reason': '

In [38]:
meetings[0]['status']

'active'

In [39]:
meetings[0]['data']

{'status_transition': [{'to': 'joining',
   'from': 'requested',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:13.141719'},
  {'to': 'awaiting_admission',
   'from': 'joining',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:22.779233'},
  {'to': 'active',
   'from': 'awaiting_admission',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:53.185960'}]}

In [32]:
meetings[2]['data']

{'completion_reason': 'stopped',
 'status_transition': [{'to': 'joining',
   'from': 'requested',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:12.938024'},
  {'to': 'awaiting_admission',
   'from': 'joining',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:22.197280'},
  {'to': 'active',
   'from': 'awaiting_admission',
   'source': 'bot_callback',
   'timestamp': '2025-09-19T18:08:48.353707'},
  {'to': 'completed',
   'from': 'active',
   'reason': 'left_alone_timeout',
   'source': 'user',
   'exit_code': 0,
   'timestamp': '2025-09-19T18:09:30.447235',
   'completion_reason': 'stopped'}]}

In [20]:
get_transcript(meeting_urls[0])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,3.00,"He didn't come down, that's why he didn't come.",en,None,Dmitriy Grankin,2025-09-19T17:31:49.081584+00:00,2025-09-19T17:31:52.081584+00:00
1,3.00,4.00,And now...,en,None,Dmitriy Grankin,2025-09-19T17:31:52.081584+00:00,2025-09-19T17:31:53.081584+00:00
2,3.30,6.30,And now it was released and it happened.,en,None,Dmitriy Grankin,2025-09-19T17:31:52.381584+00:00,2025-09-19T17:31:55.381584+00:00


KeyboardInterrupt: 

In [21]:
get_transcript(meeting_urls[1])

'absolute_start_time'


In [22]:
get_transcript(meeting_urls[2])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,1.02,6.54,"Поэтому он и не пришел, а теперь его пустили и он пришел.",ru,None,Dmitriy Grankin,2025-09-19T17:31:50.166543+00:00,2025-09-19T17:31:55.686543+00:00


KeyboardInterrupt: 

In [33]:
meetings

[{'id': 600,
  'user_id': 264,
  'platform': 'google_meet',
  'native_meeting_id': 'wjf-bhmw-bix',
  'constructed_meeting_url': 'https://meet.google.com/wjf-bhmw-bix',
  'status': 'requested',
  'bot_container_id': '42ceb1006d4b486de866b58342ab6b4a1423b28d2fc269d6b4912b33ff6c462a',
  'start_time': None,
  'end_time': None,
  'data': {},
  'created_at': '2025-09-19T17:25:12.339940',
  'updated_at': '2025-09-19T17:25:12.651872'},
 {'id': 599,
  'user_id': 264,
  'platform': 'google_meet',
  'native_meeting_id': 'fjp-cbnn-hug',
  'constructed_meeting_url': 'https://meet.google.com/fjp-cbnn-hug',
  'status': 'completed',
  'bot_container_id': '3e3af2f1ddb76f01f033c9c8cca97f28a249a46b3a364fdf5697d33be955f046',
  'start_time': None,
  'end_time': '2025-09-19T17:25:12.663591',
  'data': {'status_transition': {'to': 'joining',
    'from': 'requested',
    'source': 'bot_callback',
    'timestamp': '2025-09-19T17:25:10.027641'}},
  'created_at': '2025-09-19T17:25:07.042147',
  'updated_at': '20

In [43]:
for m in meeting_urls:
    try:stop_bot(m)
    except Exception as e:
        print(e)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:13:01 GMT, Fri, 19 Sep 2025 18:13:01 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '40', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Meeting already completed."}...

DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JbCm18ZyVsB076jQi2f2wjX1mHRjsMgFhMAPmBh8'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 18:13:01 GMT, Fri, 19 Sep 2025 18:13:01 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '40', 'content-type': 'application/json'}
DEBUG: Response conte

In [16]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['completed', 'completed', 'completed']


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's8uOUuYKTvv6vBtpWIrT3Ac9skRniMVAsSXmHHzm'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 19 Sep 2025 16:52:11 GMT, Fri, 19 Sep 2025 16:52:11 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1505', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":570,"user_id":251,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"stopping","bot_container_id":"4ab0c7294d9e9ee439c105bd7c181309e11a5f28bc2cf5e426c1015cb27a2df9","start_time":null,"end_time":null,"data":{"status_transition":{"to":"stopping","from":"requested","source":"user","timestamp":"2025-09-19T16:52:12.292814"}},"created_at":"2025-09-19T16:52:11.982282","updated_at":"2025-09-19T16:52:12...


False